## Базовая функциональность

In [78]:
import numpy as np
import pandas as pd

### Переиндексация
Для объектов pandas важен метод `reindex`, т.е. возможность создания нового объекта, в котором данные переупорядочены в соответствии с новым индексом.

In [79]:
obj = pd.Series([4.5, 7.2, -5.3, 3.6], index=['d', 'b', 'a', 'c'])
print(obj, '\n')

obj2 = obj.reindex(['a', 'b', 'c', 'd', 'e'])
print(obj2)

d    4.5
b    7.2
a   -5.3
c    3.6
dtype: float64 

a   -5.3
b    7.2
c    3.6
d    4.5
e    NaN
dtype: float64


Для упорядоченных данных, например временных рядов, иногда желательно произвести интерполяцию, или восполнение отсутствующих значений в процессе переиндексации. Это позволяет сделать параметр `method`; так, если задать для него значение `ffill`, то вместо отсутствующих значений будут подставлены значения из ближайшей предшествующей строки, имеющейся в индексе:

In [80]:
obj3 = pd.Series(['blue', 'red', 'green'], index=[0, 2, 4])
print(obj3, '\n')
obj4 = obj3.reindex(np.arange(6), method='ffill')
print(obj4)

0     blue
2      red
4    green
dtype: object 

0     blue
1     blue
2      red
3      red
4    green
5    green
dtype: object


В случае объекта DataFrame метод `reindex` может изменять строки, столбцы или то и другое. Если передать ему просто последовательность, то в результирующем объекте переиндексируются строки:

In [81]:
frame = pd.DataFrame(np.arange(9).reshape(3, 3), 
                     index=['a', 'c', 'd'], 
                     columns=['Ohio', 'Texas', 'California'])

frame 

,Ohio,Texas,California
a,0,1,2
c,3,4,5
d,6,7,8


In [82]:
frame2 = frame.reindex(index=['a', 'b', 'c', 'd'])
frame2

,Ohio,Texas,California
a,0.0,1.0,2.0
b,NaN,NaN,NaN
c,3.0,4.0,5.0
d,6.0,7.0,8.0


Столбцы можно переиндексировать, задав ключевое слово `columns`:

In [83]:
states = ['Texas', 'Utah', 'California']
frame.reindex(columns=states)

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8


Другой способ переиндексации по конкретной оси:

In [84]:
frame.reindex(states, axis='columns') 

,Texas,Utah,California
a,1,NaN,2
c,4,NaN,5
d,7,NaN,8



| Параметр | Описание |
|----------|----------|
| **labels** | То же, что и `index` (устаревшее) |
| **index** | Использовать переданную последовательность в качестве новых меток индекса |
| **columns** | Использовать переданную последовательность в качестве новых меток столбцов |
| **axis** | По какой оси переиндексировать: "index" (строки) или "columns" (столбцы). По умолчанию подразумевается "index". Можно чередовать reindex(index=new_labels) и reindex(columns=new_labels) |
| **method** | Метод интерполяции (восполнения): "ffill" (прямое восполнение – по предыдущей строке) или "bfill" (прямое восполнение – по следующей строке) |
| **fill_value** | Значение, которое должно подставляться вместо отсутствующих значений, появляющихся в результате переиндексации. Если нужно, чтобы для отсутствующих меток принималось значение NaN, то задавайте fill_value="missing" |
| **limit** | При прямом или обратном восполнении максимальная длина восполняемой лакуны (выраженная числом элементов) |
| **tolerance** | При прямом или обратном восполнении максимальная длина восполняемой лакуны (выраженная абсолютным расстоянием) |
| **level** | Сопоставить с простым объектом Index на указанном уровне MultiIndex, иначе выбрать подмножество |
| **copy** | Если True, то всегда копировать данные, даже если новый индекс эквивалентен старому. Если False, то не копировать данные, когда индексы эквивалентны |


*Лакуна - непрерывный участок отсутствующих данных, который нужно обработать*

Еще один популярный способ переиндексировать - с помощью оператора `loc`. Он работает, только если все метки нового индекса уже присутствуют в объекте DataFrame (тогда как `reindex` вставляет отсутствующие данные для новых меток): 

In [85]:
frame.loc[['a', 'd', 'c'], ['California', 'Texas']]

,California,Texas
a,2,1
d,8,7
c,5,4


### Удаление элементов из оси
Чтобы удалить один или несколько элементов из оси без использования сложных конструкций, стоит использовать `drop`. Этот метод возвращает новый объект, в котором указанные значения удалены из оси:

In [86]:
obj = pd.Series(np.arange(5), index=list('abcde'))
obj

a    0
b    1
c    2
d    3
e    4
dtype: int64

In [87]:
new_obj = obj.drop(index=['c', 'd'])
new_obj

a    0
b    1
e    4
dtype: int64

В случае DataFrame указанные в аргументе `index` значения можно удалить из любой оси:

In [88]:
data = pd.DataFrame(np.arange(16).reshape(4, 4), 
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


Если вызвать `drop`, указав последовательность меток, то будут удалены строки с такими метками.  
Чтобы удалить столбцы, используйте именованный аргумент `columns`.  
Столбцы можно удалить также, передав параметры `axis=1` (как в NumPy) или `axis="columns"`.

### Доступ по индексу, выборка и фильтрация
Доступ по индексу к объекту Series (`obj[...]`) работает так же, как и для массивов NumPy:

In [89]:
obj = pd.Series(np.arange(4), index=list('abcd'))
obj

a    0
b    1
c    2
d    3
dtype: int64

In [90]:
print('.b => ', obj.b)
print("[c] => ", obj['c'])
print("['b', 'a', 'd'] =>\n", obj[['b', 'a', 'd']])
print('[1, 3] =>\n', obj.iloc[[1, 3]])
print('[obj < 2] =>\n', obj[obj < 2])

.b =>  1
[c] =>  2
['b', 'a', 'd'] =>
 b    1
a    0
d    3
dtype: int64
[1, 3] =>
 b    1
d    3
dtype: int64
[obj < 2] =>
 a    0
b    1
dtype: int64



*Предпочтительнее выбирать значения из индекса с помощью специальных операторов `loc` и `iloc`.*

In [91]:
obj.loc[['a', 'b', 'c']] # loc для меток

a    0
b    1
c    2
dtype: int64

In [92]:
obj.iloc[[0, 1, 2]] # iloc для числовых индексов

a    0
b    1
c    2
dtype: int64

С помощью меток можно также производить вырезание, но оно отличается от обычного вырезания в Python тем, что **конечная точка включается**:

In [93]:
obj.loc['b' : 'c']

b    1
c    2
dtype: int64

Обращение по индексу к DataFrame предназначено для получения одного или нескольких столбцов путем задания одного значения или последовательности:

In [94]:
data = pd.DataFrame(np.arange(16).reshape(4, 4),
                    index=['Ohio', 'Colorado', 'Utah', 'New York'],
                    columns=['one', 'two', 'three', 'four'])
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [95]:
data['two']

Ohio         1
Colorado     5
Utah         9
New York    13
Name: two, dtype: int64

In [96]:
data[['three', 'one']]

,three,one
Ohio,2,0
Colorado,6,4
Utah,10,8
New York,14,12


In [97]:
data[:2]

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7


In [98]:
data[data['three'] > 5]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


### Выборка из DataFrame с помощью `loc` и `iloc`
Как и Series, объект DataFrame имеют специальные атрибуты `loc` и `iloc` для индексирования метками и целыми числами соответственно. Можно выбрать подмножество и строк, и столбцов одновременно.

In [99]:
data

,one,two,three,four
Ohio,0,1,2,3
Colorado,4,5,6,7
Utah,8,9,10,11
New York,12,13,14,15


In [100]:
data.loc['Colorado']

one      4
two      5
three    6
four     7
Name: Colorado, dtype: int64

In [101]:
data.loc[['Colorado', 'Utah']]

,one,two,three,four
Colorado,4,5,6,7
Utah,8,9,10,11


Оператор `loc` позволяет выбрать одновременно строки и столбцы:

In [102]:
data.loc['Colorado', ['two', 'three']]

two      5
three    6
Name: Colorado, dtype: int64

Аналогично работает и `iloc`:

In [103]:
data.iloc[[1, 2], [3, 0, 1]]

,four,one,two
Colorado,7,4,5
Utah,11,8,9


Обе функции индексирования работают не только с одиночными метками ил списками меток, но и со срезами:

In [104]:
data.loc[:'Utah', 'two']

Ohio        1
Colorado    5
Utah        9
Name: two, dtype: int64

In [105]:
data.iloc[:, :3][data.three > 5]

,one,two,three
Colorado,4,5,6
Utah,8,9,10
New York,12,13,14


#### Варианты доступа к объекту DataFrame
| Метод | Описание |
|-------|----------|
| **df[column]** | Выбрать один столбец или последовательность столбцов из DataFrame. Частные случаи: булев массив (фильтрация строк), срез (вырезание строк) или булев DataFrame (установка значений в позициях, удовлетворяющих некоторому критерию) |
| **df.loc[rows]** | Выбрать одну строку или подмножество строк из DataFrame по метке |
| **obj.loc[:, cols]** | Выбрать один столбец или подмножество столбцов по метке |
| **df.iloc[rows, cols]** | Выбрать строки и столбцы по целочисленной позиции |
| **df.iloc[rows]** | Выбрать одну строку или подмножество строк из DataFrame по целочисленной позиции |
| **obj.iloc[:, cols]** | Выбрать один столбец или подмножество столбцов по целочисленной позиции |
| **df.loc[rows, cols]** | Выбрать строки и столбцы по метке |
| **df.at[row, col]** | Выбрать одно скалярное значение по меткам строки и столбца |
| **df.iat[row, col]** | Выбрать одно скалярное значение по целочисленным позициям строки и столбца |
| **reindex()** | Выбрать строки или столбцы по меткам |

### Арифметические операции и выравнивание данных
pandas может сильно упростить работу с объектами, имеющими различные индексы. Так, если при сложении двух объектов в одном индексе обнаруживается значение, отсутствующее в другом, то результирующий индекс будет объединением исходных.

In [106]:
s1 = pd.Series([7.3, -2.5, 3.4, 1.5], index=['a', 'c', 'd', 'e'])
s2 = pd.Series([-2.1, 3.6, -1.5, 4., 3.1], index=['a', 'c', 'e', 'f', 'g'])

print(s1, '\n------------\n', s2)

a    7.3
c   -2.5
d    3.4
e    1.5
dtype: float64 
------------
 a   -2.1
c    3.6
e   -1.5
f    4.0
g    3.1
dtype: float64


In [107]:
s1 + s2

a    5.2
c    1.1
d    NaN
e    0.0
f    NaN
g    NaN
dtype: float64

Вследствие внутреннего выравнивания данных образуются значения `NaN` в позициях, для которых не нашлось соответственной пары. Эти значения распространяются на последующие операции.

В случае DataFrame выравнивание производится как для строк, так и длля столбцов:

In [108]:
df1 = pd.DataFrame(np.arange(9).reshape(3, 3), 
                   columns=['b', 'c', 'd'],
                   index=['Ohio', 'Texas', 'Colorado'])

df2 = pd.DataFrame(np.arange(12).reshape(4, 3),
                   columns=['b', 'd', 'e'],
                   index=['Utah', 'Ohio', 'Texas', 'Oregon'])

print(df1, '\n------------\n', df2)

          b  c  d
Ohio      0  1  2
Texas     3  4  5
Colorado  6  7  8 
------------
         b   d   e
Utah    0   1   2
Ohio    3   4   5
Texas   6   7   8
Oregon  9  10  11


При сложении этих объектов получается DataFrame, индекс и столбцы которого являются объединениями индексов и столбцов слагаемых:

In [109]:
df1 + df2

,b,c,d,e
Colorado,NaN,NaN,NaN,NaN
Ohio,3.0,NaN,6.0,NaN
Oregon,NaN,NaN,NaN,NaN
Texas,9.0,NaN,12.0,NaN
Utah,NaN,NaN,NaN,NaN


Поскольку столбцы `c` и `e` не встречаются в обоих DataFrame сразу, то в результирующем объекте они помечены как отсутствующие. То же самое относится к строкам и меткам, не встречающихся в обоих объектах.  
Если сложить объекты DataFrame, не имеющие обоих столбцов и строк, то результат будет содержать только значения `NaN`:

In [110]:
df1 = pd.DataFrame({'A': [1, 2]})
df2 = pd.DataFrame({'B': [3, 4]})

df1 + df2

,A,B
0,NaN,NaN
1,NaN,NaN


#### Восполнение значений в арифметических методах
При выполнении арифметических операций с объектами, проиндексированными по-разному, иногда желательно поместить специальное значение, например 0, в позиции операнда, которым в другом операнде соответствует отсутствующая позиция. 

In [111]:
df1 = pd.DataFrame(np.arange(12).reshape(3, 4), 
                   columns=list('abcd'))
df2 = pd.DataFrame(np.arange(20).reshape(4, 5), 
                   columns=list('abcde'))
df2.loc[1, 'b'] = np.nan

print(df1, '\n------------\n', df2)

   a  b   c   d
0  0  1   2   3
1  4  5   6   7
2  8  9  10  11 
------------
     a     b   c   d   e
0   0   1.0   2   3   4
1   5   NaN   7   8   9
2  10  11.0  12  13  14
3  15  16.0  17  18  19


In [112]:
df1 + df2

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,NaN
1,9.0,NaN,13.0,15.0,NaN
2,18.0,20.0,22.0,24.0,NaN
3,NaN,NaN,NaN,NaN,NaN


Чтобы заполнить все `NaN` как некоторые значения, нужно использовать `fill_value`:

In [113]:
df1.add(df2, fill_value=0)

,a,b,c,d,e
0,0.0,2.0,4.0,6.0,4.0
1,9.0,5.0,13.0,15.0,9.0
2,18.0,20.0,22.0,24.0,14.0
3,15.0,16.0,17.0,18.0,19.0


В таблице ниже представлены методы объектов Series и DataFrame. У каждого из них имеется вариант, начинающийся буквой **r** (`a.sub(b) == b.rsub(a) == a - b`), в котором аргументы переставлены местами. Например следующие операции дадут один и тот же результат:  
`1 / df1 == df1.rdiv(1)`

| Метод | Описание | Оператор |
|-------|----------|----------|
| **add**, **radd** | Сложение | `+` |
| **sub**, **rsub** | Вычитание | `-` |
| **div**, **rdiv** | Деление | `/` |
| **floordiv**, **rfloordiv** | Деление с отсечением | `//` |
| **mul**, **rmul** | Умножение | `*` |
| **pow**, **rpow** | Возведение в степень | `**` |

Точно так же, выполняя переиндексацию объекта Series или DataFrame, можно указать восполняемое значение:

In [114]:
df1.reindex(columns=df2.columns, index=df2.index, fill_value=0)

,a,b,c,d,e
0,0,1,2,3,0
1,4,5,6,7,0
2,8,9,10,11,0
3,0,0,0,0,0


#### Операции между DataFrame и Series
Как и в случае с массивами NumPy, арифметические операции между DataFrame и Series корректно определены. 

In [115]:
arr = np.arange(12).reshape(3, 4)
arr

array([[ 0,  1,  2,  3],
       [ 4,  5,  6,  7],
       [ 8,  9, 10, 11]])

In [116]:
arr - arr[0]

array([[0, 0, 0, 0],
       [4, 4, 4, 4],
       [8, 8, 8, 8]])

In [117]:
frame = pd.DataFrame(np.arange(12).reshape(4, 3),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])
print(frame)

series = frame.iloc[0]
print('-' * 12)
print(series)

        b   d   e
Utah    0   1   2
Ohio    3   4   5
Texas   6   7   8
Oregon  9  10  11
------------
b    0
d    1
e    2
Name: Utah, dtype: int64


In [118]:
frame - series

,b,d,e
Utah,0,0,0
Ohio,3,3,3
Texas,6,6,6
Oregon,9,9,9


Если какой-нибудь индекс не найден либо в столбцах DataFrame, либо в индексе Series, то объекты переиндексируются для образования объединения:

In [119]:
series2 = pd.Series(np.arange(3), index=['b', 'e', 'f'])
series2

b    0
e    1
f    2
dtype: int64

In [120]:
frame.add(series2)

,b,d,e,f
Utah,0.0,NaN,3.0,NaN
Ohio,3.0,NaN,6.0,NaN
Texas,6.0,NaN,9.0,NaN
Oregon,9.0,NaN,12.0,NaN


Если вы хотите вместо этого сопоставлять строки, а  укладывать столбцы, то должны будете воспользоваться каким-нибудь арифметическим методом, указав сопоставление по индексу:

In [121]:
series3 = frame['d']
frame.sub(series3, axis='index')

,b,d,e
Utah,-1,0,1
Ohio,-1,0,1
Texas,-1,0,1
Oregon,-1,0,1


### Применение функций и отображение
Универсальные функции NumPy отлично работают и с объектами pandas:

In [122]:
frame = pd.DataFrame(np.random.standard_normal(12).reshape(4, 3),
                     columns=list('bde'),
                     index=['Utah', 'Ohio', 'Texas', 'Oregon'])

frame

,b,d,e
Utah,-0.677651,-1.360095,-2.407849
Ohio,1.050342,0.352019,0.911205
Texas,1.538738,0.344861,-1.068105
Oregon,1.241422,-0.919012,-2.751286


In [123]:
np.abs(frame)

,b,d,e
Utah,0.677651,1.360095,2.407849
Ohio,1.050342,0.352019,0.911205
Texas,1.538738,0.344861,1.068105
Oregon,1.241422,0.919012,2.751286


Еще одна часто встречающаяся операция - применение функции, определенной для одномерных массивов, к каждому столбцу или строке. Именно это делает метод `apply` объекта DataFrame:

In [124]:
def f1(x):
    return x.max() - x.min()

frame.apply(f1)

b    2.216389
d    1.712113
e    3.662490
dtype: float64

Если передать методу `apply` аргумент `axis='columns'`, то функция применится к каждой строке:

In [125]:
frame.apply(f1, axis='columns')

Utah      1.730197
Ohio      0.698324
Texas     2.606843
Oregon    3.992708
dtype: float64

*Многие из наиболее распространенных статистик массивов (например, `sum` и `mean`) - методы DataFrame, поэтому применять `apply` к таковым не нужно.*

Функция, передаваемая методу apply, не обязана возвращать скалярное значение, она может вернуть и объект, содержащий несколько значений:

In [126]:
def f2(x):
    return pd.Series([x.min(), x.max()], index=['min', 'max'])

frame.apply(f2)

,b,d,e
min,-0.677651,-1.360095,-2.751286
max,1.538738,0.352019,0.911205


Можно использовать и поэлементные функции Python. Допустим, требуется вычислить форматированную строку для каждого элемента frame с плавающей точкой. Это позволяет сделать метод `map`:


In [127]:
def my_format(x):
    return f'{x:.2f}'

frame.map(my_format)

,b,d,e
Utah,-0.68,-1.36,-2.41
Ohio,1.05,0.35,0.91
Texas,1.54,0.34,-1.07
Oregon,1.24,-0.92,-2.75


### Сортировка и ранжирование
Для лексикографической сортировки по индексу служит метод `sort_index`, который возвращает новый отсортированный объект:

In [128]:
obj = pd.Series(np.arange(4), index=['d', 'a', 'b', 'c'])
obj

d    0
a    1
b    2
c    3
dtype: int64

In [129]:
obj.sort_index()

a    1
b    2
c    3
d    0
dtype: int64

В случае DataFrame можно сортировать по индексу, ассоциированному с любой осью:

In [130]:
frame = pd.DataFrame(np.arange(8).reshape(2, 4),
                     index=['three', 'one'],
                     columns=['d', 'a', 'b', 'c'])
frame

,d,a,b,c
three,0,1,2,3
one,4,5,6,7


In [131]:
frame.sort_index()

,d,a,b,c
one,4,5,6,7
three,0,1,2,3


In [132]:
frame.sort_index(axis='columns')

,a,b,c,d
three,1,2,3,0
one,5,6,7,4


Можно отсортировать данные по убыванию:

In [133]:
frame.sort_index(axis='columns', ascending=False)

,d,c,b,a
three,0,3,2,1
one,4,7,6,5


Для сортировки Series по значению служит метод `sort_values`:

In [134]:
obj = pd.Series([4, np.nan, 7, np.nan, -3, 2])
obj.sort_values()

4   -3.0
5    2.0
0    4.0
2    7.0
1    NaN
3    NaN
dtype: float64

Значения NaN можно поместить в начало, а не в конец с помощью параметра `na_position`: 

In [135]:
obj.sort_values(na_position='first')

1    NaN
3    NaN
4   -3.0
5    2.0
0    4.0
2    7.0
dtype: float64

Объект DataFrame можно сортировать по значениям в одном или нескольких столбцах:

In [136]:
frame = pd.DataFrame({'b' : [4, 7, -3, 2], 'a' : [0, 1, 0, 1]})

frame

,b,a
0,4,0
1,7,1
2,-3,0
3,2,1


Для сортировки по нескольким столбцам следует передать список имен: 

In [137]:
frame.sort_values(['a', 'b'])


,b,a
2,-3,0
0,4,0
3,2,1
1,7,1


#### Ранжирование
Ранжированине заключается в присваивании рангов - от единицы до длины массива, начиная с наименьшего значения. Для ранжирования применяется метод `rank` объектов Series и DataFrame; по умолчанию `rank` обрабатывает равные ранги, присваивая каждоый группе средний ранг:

In [138]:
obj = pd.Series([7, -5, 7, 4, 2, 0, 4]) 
# Здесь элементы с индексами 3 и 6 имеют одинаковые значения, 
# поэтому они получают среднее между 4 и 5 (их номера по порядку ранжирования) 
obj.rank()

0    6.5
1    1.0
2    6.5
3    4.5
4    3.0
5    2.0
6    4.5
dtype: float64

Ранги также можно присваивать в соответствии с порядком появления в данных:

In [139]:
obj.rank(method='first')

0    6.0
1    1.0
2    7.0
3    4.0
4    3.0
5    2.0
6    5.0
dtype: float64

Ранжировать DataFrame можно как по строкам, так и по столбцам:

In [140]:
frame = pd.DataFrame({
    'b': [4.3, 7, -3, 2], 
    'a' : [0, 1, 0, 1], 
    'c' : [-2, 5, 8, -2.5]})
print(frame)
print(frame.rank())
print(frame.rank(axis='columns'))

     b  a    c
0  4.3  0 -2.0
1  7.0  1  5.0
2 -3.0  0  8.0
3  2.0  1 -2.5
     b    a    c
0  3.0  1.5  2.0
1  4.0  3.5  3.0
2  1.0  1.5  4.0
3  2.0  3.5  1.0
     b    a    c
0  3.0  2.0  1.0
1  3.0  1.0  2.0
2  1.0  2.0  3.0
3  3.0  2.0  1.0


### Индексы по осям с повторяющимися значениями
Хотя для многих функций pandas требуется уникальность меток, в общем случае это необязательно.

In [141]:
obj = pd.Series(np.arange(5), index=['a', 'a', 'b', 'b', 'c'])
obj

a    0
a    1
b    2
b    3
c    4
dtype: int64

О уникальности меток можно узнать с помощью свойства `is_unique`:

In [142]:
obj.index.is_unique

False

Выборка данных - одна из основных операций, поведение которых меняется в завсимости от наличия или отсутствия дубликатов. Если метка встречается несколько раз, то возвращается **объект Series**, а если только один раз, то **скалярное значение**.

In [143]:
print(obj['a'])
print(obj['c'])

a    0
a    1
dtype: int64
4


Такое же правило действует для доступа к строкам или столбцам в DataFrame:

In [144]:
df = pd.DataFrame(np.random.standard_normal((5, 3)), 
                  index=list('aabbc'))
df

,0,1,2
a,0.250430,1.570328,0.934177
a,-0.599596,1.411699,1.790113
b,-0.694534,1.378929,-0.424819
b,1.313733,-0.390168,-0.245099
c,-1.161958,-0.052815,0.910859


In [150]:
print(df.loc['a'])
print('---------')
print(df.loc['c'])

          0         1         2
a  0.250430  1.570328  0.934177
a -0.599596  1.411699  1.790113
---------
0   -1.161958
1   -0.052815
2    0.910859
Name: c, dtype: float64


### Редукция и вычисление описательных статистик

Объекты pandas оснащены набором стандартных математических и статистических методов. Большая их часть попадает в категорию *редукций* - методов, которые вычисляют единственное значение (например сумму, среднее) для объекта Series или объект Series для строк или столбцов DataFrame:

In [151]:
df = pd.DataFrame([[1.4, np.nan], [7.1, -4.5], [np.nan, np.nan], [0.75, -1.3]],
                  index=['a', 'b', 'c', 'd'],
                  columns=['one', 'two'])
df

,one,two
a,1.40,NaN
b,7.10,-4.5
c,NaN,NaN
d,0.75,-1.3


In [153]:
df.sum(axis='columns')

a    1.40
b    2.60
c    0.00
d   -0.55
dtype: float64

*Если строка или столбец целиком состоит из значений NaN, то сумма равна 0. В противном случае значения NaN пропускаются при вычислении суммы. Этот режим можно изменить, задав параметр skipna=False, тогда при наличии хотя бы одного значения NaN в строке или в столбце сумма будет равна NaN:*

In [155]:
df.sum(axis='columns', skipna=False)

a     NaN
b    2.60
c     NaN
d   -0.55
dtype: float64

Некоторым агрегатным функциям, например mean, для получения результата требуется, чтобы по крайней мере одно значение было отлично от NaN, поэтому в данном случае мы имеем:

In [158]:
df.mean(axis='columns')

a    1.400
b    1.300
c      NaN
d   -0.275
dtype: float64

Некоторые методы, например `idxmin` и `idxmax`, возвращают косвенные статистики (индекс, при котором достигается минимум или максимум):

In [162]:
df.idxmax()

one    b
two    d
dtype: object

In [166]:
df.cumsum()

,one,two
a,1.40,NaN
b,8.50,-4.5
c,NaN,NaN
d,9.25,-5.8


Наконец, существуют методы, не относящиеся ни к редуцирующим, ни к аккумулирующим. Примером может служить метод describe, который возвращает несколько сводных статистик за одно обращение:

In [167]:
df.describe()

,one,two
count,3.000000,2.000000
mean,3.083333,-2.900000
std,3.493685,2.262742
min,0.750000,-4.500000
25%,1.075000,-3.700000
50%,1.400000,-2.900000
75%,4.250000,-2.100000
max,7.100000,-1.300000


В случае нечисловых данных `describe` возвращает другие сводные статистики:

In [168]:
obj = pd.Series(['a', 'a', 'b', 'c'] * 4)
obj.describe()

count     16
unique     3
top        a
freq       8
dtype: object

| Метод | Описание |
|-------|----------|
| `count` | Количество значений, исключая отсутствующие |
| `describe` | Вычисляет набор сводных статистик |
| `min`, `max` | Вычисляет минимальное или максимальное значение |
| `argmin`, `argmax` | Вычисляет позиции в индексе (целые числа), в которых достигается минимальное или максимальное значение соответственно |
| `idxmin`, `idxmax` | Вычисляет метки индекса, для которых достигается минимальное или максимальное значение соответственно |
| `quantile` | Вычисляет выборочный квантиль в диапазоне от 0 до 1 (по умолчанию 0.5) |
| `sum` | Сумма значений |
| `mean` | Среднее значение |
| `median` | Медиана (50%-ный квантиль) |
| `mad` | Среднее абсолютное отклонение от среднего |
| `prod` | Произведение значений |
| `var` | Выборочная дисперсия |
| `std` | Выборочное стандартное отклонение |
| `skew` | Асимметрия (третий момент) |
| `kurt` | Куртозис (четвертый момент) |
| `cumsum` | Нарастающая сумма |
| `cummin`, `cummax` | Нарастающий минимум или максимум соответственно |
| `cumprod` | Нарастающее произведение |
| `diff` | Первая арифметическая разность (полезно для временных рядов) |
| `pct_change` | Вычисляет процентное изменение |

#### Корреляция и ковариация
Некоторые статистики вычисляются по парам аргументов. Рассмотрим объекты DataFrame, содержащие цены акций и объемы биржевых сделок с сайта "Yahoo! Finance":
 

In [170]:
price = pd.read_pickle('..\examples\yahoo_price.pkl')
volume = pd.read_pickle('..\examples\yahoo_volume.pkl')


<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
<>:1: SyntaxWarning: invalid escape sequence '\e'
<>:2: SyntaxWarning: invalid escape sequence '\e'
C:\Users\as\AppData\Local\Temp\ipykernel_9644\2859895768.py:1: SyntaxWarning: invalid escape sequence '\e'
  price = pd.read_pickle('..\examples\yahoo_price.pkl')
C:\Users\as\AppData\Local\Temp\ipykernel_9644\2859895768.py:2: SyntaxWarning: invalid escape sequence '\e'
  volume = pd.read_pickle('..\examples\yahoo_volume.pkl')


Теперь вычислим процентное изменение цен:

In [171]:
returns = price.pct_change()
returns.tail()

,AAPL,GOOG,IBM,MSFT
Date,,,,
2016-10-17,-0.000680,0.001837,0.002072,-0.003483
2016-10-18,-0.000681,0.019616,-0.026168,0.007690
2016-10-19,-0.002979,0.007846,0.003583,-0.002255
2016-10-20,-0.000512,-0.005652,0.001719,-0.004867
2016-10-21,-0.003930,0.003011,-0.012474,0.042096


Метод `corr` объекта Series вычисляет корреляцию отличных от NaN, выровненных по индексу значений в двух объектах Series. Метод `cov` вычисляет ковариацию.

In [172]:
returns['MSFT'].corr(returns['IBM'])

np.float64(0.49976361144151155)

In [173]:
returns['MSFT'].cov(returns['IBM'])

np.float64(8.870655479703546e-05)

Для объекта DataFrame методы `corr` и `cov` возвращают полную корреляционную и ковариационную матрицу в виде DataFrame:

In [174]:
returns.corr()

,AAPL,GOOG,IBM,MSFT
AAPL,1.000000,0.407919,0.386817,0.389695
GOOG,0.407919,1.000000,0.405099,0.465919
IBM,0.386817,0.405099,1.000000,0.499764
MSFT,0.389695,0.465919,0.499764,1.000000


In [175]:
returns.cov()

,AAPL,GOOG,IBM,MSFT
AAPL,0.000277,0.000107,0.000078,0.000095
GOOG,0.000107,0.000251,0.000078,0.000108
IBM,0.000078,0.000078,0.000146,0.000089
MSFT,0.000095,0.000108,0.000089,0.000215


С помощью метода `corrwith` объекта DataFrame можно вычислить попарные корреляции между столбцами или строками DataFrame и другим объектом Series или DataFrame. Если передать ему объект Series, то будет возвращен Series, содержащий значения корреляции, вычисленные для каждого столбца:

In [177]:
returns.corrwith(returns['IBM'])

AAPL    0.386817
GOOG    0.405099
IBM     1.000000
MSFT    0.499764
dtype: float64

In [178]:
returns.corrwith(volume)

AAPL   -0.075565
GOOG   -0.007067
IBM    -0.204849
MSFT   -0.092950
dtype: float64

#### Уникальные значения, счетчики значений и членство
Еще один класс методов служит для извлечения информации о значениях, хранящихся в объекте Series:

In [179]:
obj = pd.Series(["c", "a", "d", "a", "a", "b", "b", "c", "c"])
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

Метод `unique` возвращает массив уникальных объектов в Series:

In [182]:
uniques = obj.unique()
uniques

array(['c', 'a', 'd', 'b'], dtype=object)

Метод `value_counts` вычисляет объект Series, содержащий частоты встречаемости значений:

In [183]:
obj.value_counts()

c    3
a    3
b    2
d    1
Name: count, dtype: int64

Метод `isin` вычисляет булев вектор членства в множестве и может быть полезен для фильтрации набора данных относительно подмножества значений:

In [185]:
obj

0    c
1    a
2    d
3    a
4    a
5    b
6    b
7    c
8    c
dtype: object

In [186]:
mask = obj.isin(['b', 'c'])
mask

0     True
1    False
2    False
3    False
4    False
5     True
6     True
7     True
8     True
dtype: bool

In [187]:
obj[mask]

0    c
5    b
6    b
7    c
8    c
dtype: object

С `isin` тесно связан метод `Index.get_indexer`, который возвращает массив индесов, описывающий соответствие между массивом потенциально повторяющихся значений и массивом, содержщим только различные значения:

In [188]:
to_match = pd.Series(['c', 'a', 'b', 'b', 'c', 'a'])
unique_vals = pd.Series(['c', 'b', 'a'])
indices = pd.Index(unique_vals).get_indexer(to_match)
indices

array([0, 2, 1, 1, 0, 2])

Для вычисления гистограммы нескольких взаимосвязанных столбцов в DataFrame:

In [189]:
data = pd.DataFrame({
    'Qu1' : [1, 3, 4, 3, 4],
    'Qu2' : [2, 3, 1, 2, 3],
    'Qu3' : [1, 5, 2, 4, 4]
    })
data

,Qu1,Qu2,Qu3
0,1,2,1
1,3,3,5
2,4,1,2
3,3,2,4
4,4,3,4


In [190]:
data['Qu1'].value_counts().sort_index()

Qu1
1    1
3    2
4    2
Name: count, dtype: int64

Для вычисления счетчиков по всем столбцам:

In [202]:
result = data.apply(pd.Series.value_counts).fillna(0)
result

,Qu1,Qu2,Qu3
1,1.0,1.0,1.0
2,0.0,2.0,1.0
3,2.0,2.0,0.0
4,2.0,0.0,2.0
5,0.0,0.0,1.0
